# Hyperparameter Tuning

In this notebook, you'll learn how to tune the hyperparameters for a machine learning algorithm. The specific scenario in this exercise is to find the optimal hyperparameter values for training a model that predicts the species of a penguin based on observed features.

> **Citation**: The penguins dataset used in the this exercise is a subset of data collected and made available by [Dr. Kristen
Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php)
and the [Palmer Station, Antarctica LTER](https://pal.lternet.edu/), a
member of the [Long Term Ecological Research
Network](https://lternet.edu/).

## Ingest data

Run the following cell to ingest the data file you will use in this exercise. The data file will be saved in the DBFS storage for your Azure Databricks cluster.

In [0]:
%sh
rm -r /dbfs/data
mkdir /dbfs/data
wget -O /dbfs/data/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/dp-090-databricks-ml/master/data/penguins.csv

## Prepare the data
  
Now let's prepare the data for machine learning. Run the following cell to:

1. Remove any incomplete rows
2. Apply appropriate data types
3. View a random sample of the data
4. Split the data into two datasets: one for training, and another for testing.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

data = spark.read.format("csv").option("header", "true").load("/data/penguins.csv")
data = data.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"),
                          col("CulmenDepth").astype("float"),
                          col("FlipperLength").astype("float"),
                          col("BodyMass").astype("float"),
                          col("Species").astype("int")
                          )
display(data.sample(0.2))

splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Island CulmenLength CulmenDepth FlipperLength BodyMass Species Torgersen 36.7 19.3 193.0 3450.0 0 Torgersen 39.2 19.6 195.0 4675.0 0 Torgersen 42.0 20.2 190.0 4250.0 0 Torgersen 41.1 17.6 182.0 3200.0 0 Torgersen 42.5 20.7 197.0 4500.0 0 Biscoe 37.8 18.3 174.0 3400.0 0 Dream 39.5 16.7 178.0 3250.0 0 Dream 42.2 18.5 180.0 3550.0 0 Dream 40.8 18.4 195.0 3900.0 0 Dream 37.0 16.9 185.0 3000.0 0 Dream 42.3 21.2 191.0 4150.0 0 Biscoe 35.0 17.9 190.0 3450.0 0 Biscoe 36.5 16.6 181.0 2850.0 0 Biscoe 41.3 21.1 195.0 4400.0 0 Biscoe 41.1 18.2 192.0 4050.0 0 Biscoe 41.1 19.1 188.0 4100.0 0 Dream 39.6 18.1 186.0 4450.0 0 Dream 38.1 18.6 190.0 3700.0 0 Dream 40.3 18.5 196.0 4350.0 0 Biscoe 37.7 16.0 183.0 3075.0 0 Biscoe 37.8 20.0 190.0 4250.0 0 Biscoe 38.1 17.0 181.0 3175.0 0 Biscoe 39.6 20.7 191.0 3900.0 0 Torgersen 37.7 19.8 198.0 3500.0 0 Dream 37.8 18.1 193.0 3750.0 0 Biscoe 50.0 15.2 218.0 5700.0 1 Biscoe 48.4 14.6 213.0 5850.0 1 Biscoe 49.3 15.7 217.0 5850.0 1 Biscoe 45.1 14.5 207.0 5050.0 1 Biscoe 48.2 15.6 221.0 5100.0 1 Biscoe 46.5 14.8 217.0 5200.0 1 Biscoe 46.4 15.0 216.0 4700.0 1 Biscoe 47.5 14.2 209.0 4600.0 1 Biscoe 49.1 14.5 212.0 4625.0 1 Biscoe 50.0 15.9 224.0 5350.0 1 Biscoe 43.4 14.4 218.0 4600.0 1 Biscoe 47.5 14.0 212.0 4875.0 1 Biscoe 45.5 14.5 212.0 4750.0 1 Biscoe 49.5 16.1 224.0 5650.0 1 Biscoe 51.1 16.5 225.0 5250.0 1 Biscoe 55.9 17.0 228.0 5600.0 1 Biscoe 49.1 15.0 228.0 5500.0 1 Biscoe 47.3 13.8 216.0 4725.0 1 Biscoe 46.8 16.1 215.0 5500.0 1 Biscoe 53.4 15.8 219.0 5500.0 1 Biscoe 50.5 15.2 216.0 5000.0 1 Biscoe 49.8 15.9 229.0 5950.0 1 Biscoe 45.2 14.8 212.0 5200.0 1 Biscoe 49.9 16.1 213.0 5400.0 1 Dream 51.3 19.2 193.0 3650.0 2 Dream 45.4 18.7 188.0 3525.0 2 Dream 46.0 18.9 195.0 4150.0 2 Dream 52.0 18.1 201.0 4050.0 2 Dream 45.9 17.1 190.0 3575.0 2 Dream 54.2 20.8 201.0 4300.0 2 Dream 49.7 18.6 195.0 3600.0 2 Dream 49.0 19.5 210.0 3950.0 2 Dream 51.5 18.7 187.0 3250.0 2 Dream 48.1 16.4 199.0 3325.0 2 Dream 55.8 19.8 207.0 4000.0 2 Dream 49.6 18.2 193.0 3775.0 2

Training Rows: 241  Testing Rows: 101


The data consists of measurements of the following details of penguins that have been observed in Antartica:

- **Island**: The island in Antartica where the penguin was observed.
- **CulmenLength**: The length in mm of the penguin's culmen (bill).
- **CulmenDepth**: The depth in mm of the penguin's culmen.
- **FlipperLength**: The length in mm of the penguin's flipper.
- **BodyMass**: The body mass of the penguin in grams.
- **Species**: An integer value that represents the species of the penguin:
  - **0**: *Adelie*
  - **1**: *Gentoo*
  - **2**: *Chinstrap*
  
Our goal in this project is to use the observed charateristics of a pengun (its *features*) in order to predict its species (which in machine learning terminology, we call the *label*).

## Optimize hyperparameter values for training a model

You train a machine learning model by fitting the features to an algorithm that calculates the most probable label. Algorithms take the training data as a parameter and attempt to calculate a mathematical relationship between the features and labels. In addition to the data, most algorithms use one or more *hyperparameters* to influence the way the relationship is calculated; and determining the optimal hyperparameter values is an important part of the iterative model training process.

To help you determine optimal hyperparameter values, Azure Databricks includes support for **Hyperopt** - a library that enables you to try multiple hyperparameter values and find the best combination for your data.

### Create a training function

The first step in using Hyperopt is to create a function that:

- Trains a model using one or more hyperparameter values that are passed to the function as parameters.
- Calculates a performance metric that can be used to measure *loss* (how far the model is from perfect prediction performance)
- Returns the loss value so can be optimized (minimized) iteratively by trying different hyperparameter values

Run the following cell to create a function that does this for the penguins data.

In [0]:
from hyperopt import STATUS_OK
import mlflow
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def objective(params):
    # Train a model using the provided hyperparameter value
    catFeature = "Island"
    numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
    catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
    numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
    numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
    featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
    mlAlgo = DecisionTreeClassifier(labelCol="Species", featuresCol="Features",
                                    maxDepth=params['MaxDepth'], maxBins=params['MaxBins'])
    pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, mlAlgo])
    model = pipeline.fit(train)
    
    # Evaluate the model to get the target metric
    prediction = model.transform(test)
    eval = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction", metricName="accuracy")
    accuracy = eval.evaluate(prediction)
    
    # Hyperopt tries to minimize the objective function, so you must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}


### Iteratively run the function to optimize hyperparameters

Now that you have a function defined, you need to call it iteratively with different combinations of hyperparameter values.

To do this, you need to:

- Define a search space that specifies the range of values to be used for one or more hyperparameters (see [Defining a Search Space](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/) in the Hyperopt documentation for more details).
- Specify the Hyperopt algorithm you want to use (see [Algorithms](http://hyperopt.github.io/hyperopt/#algorithms) in the Hyperopt documentation for more details).
- Use the **hyperopt.fmin** function to call your training function repeatedly and try to minimize the loss.

Run the following cell to optimize hyperparameters for your penguin classification training function (this may take some time!)

In [0]:
from hyperopt import fmin, tpe, hp

# Define a search space for two hyperparameters (maxDepth and maxBins)
search_space = {
    'MaxDepth': hp.randint('MaxDepth', 10),
    'MaxBins': hp.choice('MaxBins', [10, 20, 30])
}

# Specify an algorithm for the hyperparameter optimization process
algo=tpe.suggest

# Call the training function iteratively to find the optimal hyperparameter values
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=12)

print("Best param values: ", argmin)


  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:43<07:53, 43.00s/trial, best loss: -0.9306930693069307]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 17%|█▋        | 2/12 [01:25<07:06, 42.68s/trial, best loss: -0.9702970297029703]

...

 83%|████████▎ | 10/12 [06:51<01:21, 40.73s/trial, best loss: -0.9702970297029703]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 92%|█████████▏| 11/12 [07:32<00:40, 40.87s/trial, best loss: -0.9702970297029703]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 12/12 [08:14<00:00, 41.25s/trial, best loss: -0.9702970297029703]
Best param values:  {'MaxBins': 0, 'MaxDepth': 7}


The code will iteratively run the training function 6 times (based on the **max_evals** setting).
Each run is recorded by MLflow, and you can use the the **&#9656;** toggle to expand the **MLflow run** output under the code cell and select the **experiment** hyperlink to view them. Each run is assigned a random name, and you can view each of them in the MLflow run viewer to see details of parameters and metrics that were recorded.

When all of the runs have finished, the code displays details of the best hyperparameter values that were found (the combination that resulted in the least loss). In this case, the **MaxBins** parameter is defined as a choice from a list of three possible values (10, 20, and 30) - the best value indicates the zero-based item in the list (so 0=10, 1=20, and 2=30). The **MaxDepth** parameter is defined as a random integer between 0 and 10, and the integer value that gave the best result is displayed. For more information about specifying hyperparameter value scopes for search spaces, see [Parameter Expressions](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/#parameter-expressions) in the Hyperopt documentation.

## Use the Trials class to log run details

In addition to using MLflow experiment runs to log details of each iteration, you can also use the **hyperopt.Trials** class to record and view details of each run.

In [0]:
from hyperopt import Trials

# Create a Trials object to track each run
trial_runs = Trials()

argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=12,
  trials=trial_runs)

print("Best param values: ", argmin)

# Get details from each trial run
print ("trials:")
for trial in trial_runs.trials:
    print ("\n", trial)

  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:45<08:17, 45.24s/trial, best loss: -0.9306930693069307]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 17%|█▋        | 2/12 [01:25<07:03, 42.33s/trial, best loss: -0.9504950495049505]

...

 83%|████████▎ | 10/12 [06:48<01:20, 40.40s/trial, best loss: -0.9702970297029703]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

 92%|█████████▏| 11/12 [07:29<00:40, 40.53s/trial, best loss: -0.9702970297029703]

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 12/12 [08:08<00:00, 40.72s/trial, best loss: -0.9702970297029703]
Best param values:  {'MaxBins': 0, 'MaxDepth': 4}
trials:

 {'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.9306930693069307, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'MaxBins': [0], 'MaxDepth': [0]}, 'vals': {'MaxBins': [1], 'MaxDepth': [6]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 11, 23, 0, 15, 47, 444000), 'refresh_time': datetime.datetime(2023, 11, 23, 0, 16, 32, 677000)}

 {'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.9504950495049505, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'MaxBins': [1], 'MaxDepth': [1]}, 'vals': {'MaxBins': [2], 'MaxDepth': [8]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 11, 23, 0, 16, 32, 680000), 'refresh_time': datetime.datetime

In this notebook, you've been introduced to **Hyperopt** as a way to optimize hyperparameter values for machine learning models.

For more information see [Hyperparameter tuning](https://learn.microsoft.com/azure/databricks/machine-learning/automl-hyperparam-tuning/) in the Azure Databricks documentation.

In [0]:
from hyperopt import SparkTrials

spark_trials = SparkTrials()
with mlflow.start_run():
    argmin = fmin(
        fn=objective,
        space=search_space,
        algo=algo,
        max_evals=100,
        trials=spark_trials)

print("Best param values: ", argmin)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (8), which is Spark's default parallelism (8), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.
Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.
Total Trials: 0: 0 succeeded, 0 failed, 0 cancelled.


---------------------------------------------------------------------------
PySparkRuntimeError                       Traceback (most recent call last)
File /databricks/python/lib/python3.10/site-packages/cloudpickle/cloudpickle_fast.py:602, in CloudPickler.dump(self, obj)
    601 try:
--> 602     return Pickler.dump(self, obj)
    603 except RuntimeError as e:

File /databricks/spark/python/pyspark/context.py:514, in SparkContext.__getnewargs__(self)
    512 def __getnewargs__(self) -> NoReturn:
    513     # This method is called when attempting to pickle SparkContext, which is always an error:
--> 514     raise PySparkRuntimeError(
    515         error_class="CONTEXT_ONLY_VALID_ON_DRIVER",
    516         message_parameters={},
    517     )

PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on work